In [1]:
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait # 로딩까지 기다리는
from selenium.webdriver.support import expected_conditions as EC # #예외상황이 떴을 때 대처하는 라이브러리
from selenium.webdriver.chrome.options import Options #필요한 옵션을 가져옴

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys #키보드
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import tqdm
import pyperclip
import time
import csv
import random

In [16]:
# 크롤링 시작과 끝 날짜(월별로 크롤링)
first_days = pd.date_range('2020/01/01', '2022/10/31', freq="MS") #각 월의 첫 날
last_days = pd.date_range('2020/01/01', '2022/10/31', freq="M") #각 월의 마지막 날

keyword_list = ["치매 간병 -보험", "치매 간병 힘든점 -보험", '치매 가출 -보험']

In [3]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage') #overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치


In [14]:
# 블로그 url 수집
blog_url_list = []

# 네이버에 키워드 검색
for keyword in  keyword_list:
    url = f'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={keyword}&oquery={keyword}%27'
    
    driver = webdriver.Chrome(executable_path=r"C:\Developes\python\고객분석\chromedriver", options=chrome_options)
    driver.implicitly_wait(2)
    
    driver.get(url)
    time.sleep(1)
    
    url_want = 990
    
    # 옵션 버튼
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()
    
    # 날짜 지정
    for k in range(len(first_days)):
        try:
            # 시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
       
            # 스크롤 조작
            last_heigt = driver.execute_script('return document.body.scrollHeight') # 몇 번을 스크롤해야 끝까지 내려가는지 횟수 반환
            scroll = (last_heigt/30)-1 # 한 페이지에 기본 30개의 글이 있음
            
            # 스크롤 다운
            for a in range(int(scroll)):
                # 끝까지 스크롤 다운
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1, 1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight;')
                
                if new_height == last_heigt:
                    break
                last_heigt = new_height
            
            soup = bs(driver.page_source)
            blog_url = soup.select('a.api_txt_lines.total_tit')
            
            # url주소 수집
            for b in blog_url:
                blog_url_list.append(b['href']) # 주소 수집
                # print(f'{first_days.year[k]} 키워드 {first_days.month[k]}') 
                driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass 
        
    driver.quit()



                


In [15]:
len(blog_url_list)

14

In [6]:
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
import re

no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

10개 블로그 크롤링 완료
20개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://serendifity-life.tistory.com/108
30개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://www.lifentalk.com/2221
40개 블로그 크롤링 완료
50개 블로그 크롤링 완료
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://madamamy.tistory.com/87
100개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://realtopic.tistory.com/161
네이버 블로그 아닌 url:  https://lab912.tistory.com/1119
110개 블로그 크롤링 완료
120개 블로그 크롤링 완료
130개 블로그 크롤링 완료
140개 블로그 크롤링 완료
150개 블로그 크롤링 완료
160개 블로그 크롤링 완료
170개 블로그 크롤링 완료
180개 블로그 크롤링 완료
190개 블로그 크롤링 완료
네이버 블로그 아닌 url:  https://felkmt.tistory.com/4189
200개 블로그 크롤링 완료
210개 블로그 크롤링 완료


In [7]:
len(time_list)

213

In [8]:
data = {'time' : time_list,
        'review' : review_list,
        'comment' : comment_list,
        'like' : like_list,
        'url' : url_list
        }

df = pd.DataFrame(data)

print('Duration: ', time.time() - start)
print('Total {} of blog crawling are completed'.format(count))

Duration:  1295.4784572124481
Total 213 of blog crawling are completed


In [12]:
df

,time,review,comment,like,url
0,2021. 1. 17. 19:59,DAILY 월드비전 후원 / 10주년 기념증서&후원아동 성인이...,[],18,https://blog.naver.com/yujin5093/222210442940
1,2021. 1. 25. 21:57,"기부, 나눔, 선행 후원받던 아이가 후원단체 수장됐다… “하나...",[],1,https://blog.naver.com/wgj1290/222219784291
2,2021. 1. 23. 21:01,﻿ ...,[],3,https://blog.naver.com/ch_review/222217577974
3,2021. 1. 18. 12:02,"이건 어떻게? 후원 중단 방법 (월드비전, 어린이재단, 유니세...",[],8,https://blog.naver.com/kjsunj/222211122982
4,2021. 1. 11. 22:10,어려서부터 엄마가 월드비전에 후원하시는걸 봐왔다.주기적으로 아이들에 대한 보고서와...,[],2,https://blog.naver.com/yahomilk/222204267720
...,...,...,...,...,...
208,2021. 1. 30. 16:00,"문서(희로애락) [뉴스하이킥] 이광기 ""코로나로 가족 잃은 분...",[],[],https://blog.naver.com/impear/222225354614
209,2021. 1. 5. 1:57,일상 모음집 2020년 12월 조각 모음 (上) 굴...,[],8,https://blog.naver.com/dpsk4450/222196625320
210,2021. 1. 1. 16:20,서평 DOING GOOD BETTER 능서쌀맛 ...,[],1,https://blog.naver.com/rmstn8426/222192578110
211,2021. 1. 17. 13:34,ᴛʀᴀᴠᴀɪʟ [KOICA ODA YP] 코이카 ODA 사업수...,[],11,https://blog.naver.com/shi_lver/222210141366


In [10]:
df.to_excel("월드비전.xlsx", index=True, encoding='utf-8-sig')